Goal: Examine correlations between global,socially significant events and Bitcoin blockchain metrics (block congestion, price movement, metadata notes) over time.
Utilizing publick bitcoin blockchain datasets stored in bigquery
to begin, I installed the following libraries:
pip install google-cloud-bigquery pandas pyarrow

Once I started trying to communicate with bigquery database, I had to also install:
pip install db-dtypes
to help pandas understand how to display data types used in SQL like "DATE, TIMESTAMP, STRUCT,etc.

Tables used:
 
A. bigquery-public-data.bitcoin_blockchain.blocks table
B. bigquery-public-data.bitcoin_blockchain.transactions table
C. bigquery-public-data.crypto_bitcoin.outputs table


In [1]:
%pip install db_dtypes


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from google.cloud import bigquery
import os
#Set the environement variable for key file (telling python where my login key file is)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "Credentials/capstone-project-461516-4b2a88c5f4cd.json"
#Start the BigQuery client using my key
client = bigquery.Client()



In [3]:
#Testing my code to see if it is working:
query1 = """
    SELECT block_id
    FROM `bigquery-public-data.bitcoin_blockchain.blocks`
    LIMIT 5
"""


df = client.query(query1).to_dataframe()
print(df)

                                            block_id
0  00000000000000000045fc015fe17335fdedb9ebca4b3d...
1  000000000000000000051cb4a6272786418c5357bff9d0...
2  000000000000000000d091f9c5d7815e1a685f702b4164...
3  000000000000000000235b8a4ec7a7cc12c1c9133f2d34...
4  000000000000000000719106d55fea7af59d31cacef736...


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [4]:
import sys
print(sys.executable)


c:\Python313\python.exe


In [5]:
query2 = """
    SELECT *
    FROM `bigquery-public-data.bitcoin_blockchain.blocks`
    LIMIT 5
"""
print(df.columns)

Index(['block_id'], dtype='object')


In [6]:
query3 = """
    SELECT *
    FROM `bigquery-public-data.bitcoin_blockchain.transactions`
    LIMIT 5
"""
print(df.columns)

Index(['block_id'], dtype='object')


In [7]:
query4 = """
    SELECT 
      o.script_asm,
      o.block_number,
      o.value,
      o.transaction_hash
    FROM `bigquery-public-data.crypto_bitcoin.outputs` AS o
    WHERE o.script_asm LIKE '%OP_RETURN%'
    LIMIT 10
"""

df = client.query(query4).to_dataframe()
print(df.head())


                                          script_asm  block_number value  \
0  OP_RETURN 48454d4901015b8a39001fb229e9d7ae0540...        902567  0E-9   
1    OP_RETURN 13 ff7f818cec82d08bc0a882e1aefe88bc02        902567  0E-9   
2    OP_RETURN 13 ff7f818cec82d08bc0a882e1aefe88bc02        902567  0E-9   
3  OP_RETURN 6664653a746f3a44414928425343293a3078...        902567  0E-9   
4  OP_RETURN 48454d4901015b898b001fb17a79e61ee382...        902567  0E-9   

                                    transaction_hash  
0  a56542c564454a5633e0b4330188e9e156fe7304ab979d...  
1  dff7afc4f712d58e9f7f6ed58a09d15f8586878a291001...  
2  a0cce63135aa219adecbc9bf35da31510d5bea46770953...  
3  440fc77213c660d51a65c8058f7cf8214471e941ac7b5f...  
4  9a333bf3cd9f5c8b02fc0aa1c02a09a92f3cbf5ea78971...  


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [8]:
query5 = """
    SELECT *
    FROM `bigquery-public-data.crypto_bitcoin.outputs` AS o
    WHERE o.script_asm LIKE '%OP_RETURN%'
    LIMIT 10
"""

df = client.query(query5).to_dataframe()
print(df.columns)

Index(['transaction_hash', 'block_hash', 'block_number', 'block_timestamp',
       'index', 'script_asm', 'script_hex', 'required_signatures', 'type',
       'addresses', 'value'],
      dtype='object')


C:\Users\smby2\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


Stopped here as of 5/31. when I left off, I was able to see that tables:
A. bigquery-public-data.bitcoin_blockchain.blocks table
B. bigquery-public-data.bitcoin_blockchain.transactions table
C. bigquery-public-data.crypto_bitcoin.outputs table
cointained the data I needed to connect block timestamp to messages hidden in OP_RETURN column (in the outputs table) 
1. blocks → has block_hash and block_timestamp
2. transactions → has both transaction_hash and block_hash
3. outputs → has transaction_hash



